In [102]:
import numpy as np
import pandas as pd
import itertools
import warnings
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
warnings.filterwarnings(action='ignore')

Ucitavanje dataset-a

In [72]:
df=pd.read_csv("C:/Users/38164/Master studije/Data Mining/02 DM Stabla odlucivanja/breast-cancer.data")
df.columns=['class','age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat']
new_cols=['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat','class']
df=df.reindex(columns=new_cols)
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
2,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
3,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
4,60-69,ge40,15-19,0-2,no,2,left,left_low,no,no-recurrence-events


Informacije o dataset-u:

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          285 non-null    object
 1   menopause    285 non-null    object
 2   tumor-size   285 non-null    object
 3   inv-nodes    285 non-null    object
 4   node-caps    285 non-null    object
 5   deg-malig    285 non-null    int64 
 6   breast       285 non-null    object
 7   breast-quad  285 non-null    object
 8   irradiat     285 non-null    object
 9   class        285 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.4+ KB


In [74]:
df.shape

(285, 10)

In [75]:
df.drop_duplicates()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
2,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
3,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
4,60-69,ge40,15-19,0-2,no,2,left,left_low,no,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
280,30-39,premeno,30-34,0-2,no,2,left,left_up,no,recurrence-events
281,30-39,premeno,20-24,0-2,no,3,left,left_up,yes,recurrence-events
282,60-69,ge40,20-24,0-2,no,1,right,left_up,no,recurrence-events
283,40-49,ge40,30-34,3-5,no,3,left,left_low,no,recurrence-events


Izbacivanje duplikata

In [76]:
df=df.drop_duplicates()
df.shape

(271, 10)

Izbacivanje nedostajucih vrednosti

In [77]:
df.isnull().sum()

age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      0
deg-malig      0
breast         0
breast-quad    0
irradiat       0
class          0
dtype: int64

In [78]:
features = df.drop(['class'], axis=1)
target=df.copy()
target=target.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
target.head()
test=df.drop(['class'],axis=1)
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
2,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
3,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
4,60-69,ge40,15-19,0-2,no,2,left,left_low,no,no-recurrence-events


Pomocne funkcije:

In [79]:
def gini_impurity(a=0, b=0):
    if a==b==0:
        return 0
    return 1 - np.square(a/(a+b)) - np.square(b/(a+b))

def gini_split(a=0, b=0, c=0, d=0):
    n=a+b+c+d
    gini_total=((a+b)/n)*gini_impurity(a, b)+((c+d)/n)*gini_impurity(c, d)
    return gini_total

def compute_accuracy(Y_true, Y_pred):  
    correctly_predicted = 0  
    # iterating over every label and checking it with the true sample  
    for true_label, predicted in zip(Y_true, Y_pred):  
        if true_label == predicted:  
            correctly_predicted += 1  
    # computing the accuracy score  
    accuracy_score = correctly_predicted / len(Y_true)  
    return accuracy_score 

Klasa Node koja predstavlja cvor

In [80]:
class Node:
    def __init__(self, features, target,feature_question=None, question=None, class_val=None):
        self.features=features
        self.target=target
        self.feature_question=feature_question
        self.left=None
        self.right=None
        self.question=question
        self.class_val=class_val

Klasa DecisionTree koja predstavlja model stabla odlucivanja

In [93]:
class DecisionTree:
    def __init__(self, features, target):
        self.root=None
        self.features=features
        self.target=target
        self.target_column=target.columns
        self.class_col=target.columns
        self.no=target[self.class_col[0]].unique()[0]
        self.yes=target[self.class_col[0]].unique()[1]
        print(self.target_column)
        
    def all_attributes_equal(self, data):
        agr=0
        for feature in self.features:
            if data[feature].nunique()==1:
                agr+=1
        if agr==len(self.features):
            return True
        else:
            return False
    
    def homogenous(self, data):
        if data[self.class_col[0]].nunique()==1:
            return True
        else:
            return False
    
    
    def most_valuable_split(self,features, target, data):
        combinations=[]
        best_gini=1
        best_feature=None
        best_split=None
        arguments=[]
        for feature in features.columns:
            if features[feature].nunique()==1:
                continue
            for i in range(1, features[feature].nunique()):
                combinations = combinations + list(itertools.combinations(features[feature].unique(), i))
            for item in combinations:
                n1=target[features[feature].isin(item)]
                n2=target[~features[feature].isin(item)]  
                #args=n1.value_counts().tolist()+n2.value_counts().tolist()
                if self.no in n1.values:
                    arguments.append(n1.value_counts()[self.no])
                else:
                    arguments.append(0)               
                if self.yes in n1.values:
                    arguments.append(n1.value_counts()[self.yes])
                else:
                    arguments.append(0)                
                if self.no in n2.values:
                    arguments.append(n2.value_counts()[self.no])
                else:
                    arguments.append(0)                
                if self.yes in n2.values:
                    arguments.append(n2.value_counts()[self.yes])
                else:
                    arguments.append(0)
                gini_ind=gini_split(*arguments)
                #gini_ind=gini_split(a, b, c, d)
                arguments=[]
                if best_gini>gini_ind:
                    best_gini=gini_ind
                    best_split=item   
                    best_feature=feature
            combinations=[]
        return best_split, best_feature, best_gini
    
    def decision_tree_classifier(self, features, target, data):
        if len(data.index)<3:
            self.features=features
            self.target=target
            class_val=data[self.class_col[0]].value_counts().idxmax()
            return_node1=Node(features=features, target=target, class_val=class_val)
            #print(return_node1.class_val)
            return Node(features=features, target=target, class_val=class_val)
        elif self.all_attributes_equal(data)==True:
            self.features=features
            self.target=target
            class_val=data[self.class_col[0]].value_counts().idxmax()
            return_node1=Node(features=features, target=target, class_val=class_val)
            #print(return_node1.class_val)
            return Node(features=features, target=target, class_val=class_val)
            #return Vertex(features, target, data[target].value_counts().idxmax())
        elif self.homogenous(data)==True:
            self.features=features
            self.target=target
            class_val=data[self.class_col[0]].value_counts().idxmax()
            return_node2=Node(features=features, target=target, class_val=class_val)
            #print(return_node2.class_val)
            return Node(features=features, target=target, class_val=class_val)
            #return Vertex(features, target, data[target].value_counts().idxmax())
        else:
            new_best_split, new_best_feature, new_best_gini=self.most_valuable_split(features,data[self.class_col[0]], data)
            self.question=new_best_split
            self.feature_question=new_best_feature
            self.my_gini=new_best_gini
            #new_features=features.drop([new_best_feature], axis=1)
            data_right=data[data[new_best_feature].isin(new_best_split)]
            features_right=data_right.drop([self.class_col[0]], axis=1)
            data_left=data[~data[new_best_feature].isin(new_best_split)]
            features_left=data_left.drop([self.class_col[0]], axis=1)
#           right=self.decision_tree_classifier(features_right,target, data_right)
#           left=self.decision_tree_classifier(features_left,target, data_left)
            return_node=Node(features=features, target=target, feature_question=new_best_feature, question=new_best_split)
            #print(return_node.feature_question, return_node.question, new_best_gini)
            return_node.right=self.decision_tree_classifier(features_right,target, data_right)
            return_node.left=self.decision_tree_classifier(features_left,target, data_left)
            return return_node
            
    
    def fit(self, features, target, data):
        self.root=self.decision_tree_classifier(features, target, data)
        
    def predicting(self, features, target, data, node):
        if node.class_val!=None:
            #print(node.class_val)
            return node.class_val
        feature_value=data[node.feature_question]
        if feature_value in node.question:
            right=node.right
            return self.predicting(features, target, data, right)
        if feature_value not in node.question:
            left=node.left 
            return self.predicting(features, target, data, left)
    
    def predict(self, features, target, data):
        return [self.predicting(features, target, row, self.root) for index, row in data.iterrows()]

Bez Cross Validation:

In [94]:
tree=DecisionTree(features, target)
tree.fit(features, target, df)
true_values=df['class']
predicted_values=tree.predict(features, target, test)
print(confusion_matrix(true_values, predicted_values))
print(compute_accuracy(true_values, predicted_values))

Index(['class'], dtype='object')
[[184   6]
 [ 25  56]]
0.8856088560885609


K Fold Cross Validation:

In [67]:
df = df.reindex(np.random.permutation(df.index))   
df = df.reset_index(drop=True)

fold1 = df.loc[0:55]                                            
fold2 = df.loc[56:111]
fold3 = df.loc[112:167]
fold4 = df.loc[168:223]
fold5 = df.loc[224:271]

train_val1 = pd.concat([fold1, fold2, fold3, fold4])
test_val1 = fold5
test_val1_nolabels=test_val1.drop(['class'], axis=1)
features_train_val1 = train_val1.drop(['class'], axis=1)
target_train_val1=train_val1.copy()
target_train_val1=target_train_val1.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
features_test_val1 = test_val1.drop(['class'], axis=1)
target_test_val1=test_val1.copy()
target_test_val1=target_test_val1.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)

train_val2 = pd.concat([fold1, fold2, fold3, fold5])
test_val2 = fold4
test_val2_nolabels=test_val2.drop(['class'], axis=1)
features_train_val2 = train_val2.drop(['class'], axis=1)
target_train_val2=train_val2.copy()
target_train_val2=target_train_val2.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
features_test_val2 = test_val2.drop(['class'], axis=1)
target_test_val2=test_val2.copy()
target_test_val2=target_test_val2.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)

train_val3 = pd.concat([fold1, fold2, fold4, fold5])
test_val3 = fold3
test_val3_nolabels=test_val3.drop(['class'], axis=1)
features_train_val3 = train_val3.drop(['class'], axis=1)
target_train_val3=train_val3.copy()
target_train_val3=target_train_val3.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
features_test_val3 = test_val3.drop(['class'], axis=1)
target_test_val3=test_val3.copy()
target_test_val3=target_test_val3.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)

train_val4 = pd.concat([fold1, fold3, fold4, fold5])
test_val4 = fold2
test_val4_nolabels=test_val4.drop(['class'], axis=1)
features_train_val4 = train_val4.drop(['class'], axis=1)
target_train_val4=train_val4.copy()
target_train_val4=target_train_val4.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
features_test_val4 = test_val4.drop(['class'], axis=1)
target_test_val4=test_val4.copy()
target_test_val4=target_test_val4.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)

train_val5 = pd.concat([fold2, fold3, fold4, fold5])
test_val5 = fold1
test_val5_nolabels=test_val5.drop(['class'], axis=1)
features_train_val5 = train_val5.drop(['class'], axis=1)
target_train_val5=train_val5.copy()
target_train_val5=target_train_val5.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)
features_test_val5 = test_val5.drop(['class'], axis=1)
target_test_val5=test_val5.copy()
target_test_val5=target_test_val5.drop(['age', 'menopause', 'tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat'], axis=1)

In [92]:
tree=DecisionTree(features_train_val1, target_train_val1)
tree.fit(features_train_val1, target_train_val1, train_val1)
true_values1=test_val1['class']
predicted_values1=tree.predict(features_test_val1, target_test_val1, test_val1_nolabels)
score1=compute_accuracy(true_values1, predicted_values1)
print(confusion_matrix(true_values1, predicted_values1))

tree=DecisionTree(features_train_val2, target_train_val2)
tree.fit(features_train_val2, target_train_val2, train_val2)
true_values2=test_val2['class']
predicted_values2=tree.predict(features_test_val2, target_test_val2, test_val2_nolabels)
score2=compute_accuracy(true_values2, predicted_values2)
print(confusion_matrix(true_values2, predicted_values2))

tree=DecisionTree(features_train_val3, target_train_val3)
tree.fit(features_train_val3, target_train_val3, train_val3)
true_values3=test_val3['class']
predicted_values3=tree.predict(features_test_val3, target_test_val3, test_val3_nolabels)
score3=compute_accuracy(true_values3, predicted_values3)
print(confusion_matrix(true_values3, predicted_values3))

tree=DecisionTree(features_train_val4, target_train_val4)
tree.fit(features_train_val4, target_train_val4, train_val4)
true_values4=test_val4['class']
predicted_values4=tree.predict(features_test_val4, target_test_val4, test_val4_nolabels)
score4=compute_accuracy(true_values4, predicted_values4)
print(confusion_matrix(true_values4, predicted_values4))

tree=DecisionTree(features_train_val5, target_train_val5)
tree.fit(features_train_val5, target_train_val5, train_val5)
true_values5=test_val5['class']
predicted_values5=tree.predict(features_test_val5, target_test_val5, test_val5_nolabels)
score5=compute_accuracy(true_values5, predicted_values5)
print(confusion_matrix(true_values5, predicted_values5))
                        
print('accuracy score is', (score1+score2+score3+score4+score5)/5)

Index(['class'], dtype='object')
[[30  8]
 [ 4  5]]
Index(['class'], dtype='object')
[[27  8]
 [15  6]]
Index(['class'], dtype='object')
[[32  6]
 [14  4]]
Index(['class'], dtype='object')
[[31  9]
 [10  6]]
Index(['class'], dtype='object')
[[33  6]
 [13  4]]
accuracy score is 0.6596504559270516
